# Swaption Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-swaption-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [3]:
import refinitiv.dataplatform as rdp
import pandas as pd

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
#IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Valuing an existing Bemudan Swaption

This sample value on the 2020-09-12 with market data from 2020-10-09 an existing swaption expiring on 2021-01-20 defined on a standard 6 month euribor swap. The 'fields' requested are:
- descriptive fields ("InstrumentTag","InstrumentDescription","StartDate","EndDate")
- the market value in cash ("MarketValueInDealCcy")
- the Greeks in amount ("DeltaAmountInDealCcy", "GammaAmountInDealCcy", "ThetaAmountInDealCcy", "VegaAmountInDealCcy")
- the Greeks in percent ("DeltaPercent", "GammaPercent", "ThetaPercent", "VegaPercent")
- the names of the zero-coupon curves used for discounting ("DiscountCurveName") and estimating forward rates ("ForwardCurveName")
- the error information ("ErrorCode", "ErrorMessage")


In [4]:
request_body = {
    
    "fields" : ["InstrumentTag","InstrumentDescription","StartDate","EndDate","MarketValueInDealCcy",
                "DeltaAmountInDealCcy","DeltaPercent", "GammaAmountInDealCcy", "GammaPercent", "ThetaAmountInDealCcy", "ThetaPercent", "VegaAmountInDealCcy", "VegaPercent",
                "DiscountCurveName","ForwardCurveName","ErrorCode","ErrorMessage"],
    
    "universe" : [
    {   
        "instrumentType": "Swaption",
        
        "instrumentDefinition": {
            "instrumentTag": "EUR_6M_5Y_pay",
            "settlementType": "Physical",
            "buySell": "Buy",
            "callPut": "Call",
            "exerciseStyle":"BERM",
            "NotionalAmount":1000000,
            "endDate": "2021-01-20",
            "strikePercent": 0.213736199,
            "underlyingDefinition": {
                "tenor": "5Y",
                "template": "EUR_AB6E"
            },
            
            "bermudanSwaptionDefinition" : {
                "exerciseScheduleType" : "FixedLeg"
            }
        }        
    }],
    
    "pricingParameters": {
        "marketDataDate": "2020-09-10",
        "valuationDate": "2020-09-12"
    },
        
    "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,InstrumentDescription,StartDate,EndDate,MarketValueInDealCcy,DeltaAmountInDealCcy,DeltaPercent,GammaAmountInDealCcy,GammaPercent,ThetaAmountInDealCcy,ThetaPercent,VegaAmountInDealCcy,VegaPercent,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,EUR_6M_5Y_pay,long payer BERM 4M x 5Y Swaption,2021-01-22T00:00:00Z,2026-01-22T00:00:00Z,4156.809616,113.970051,1.139701,1.54221,154.220991,5.746472,0.000006,171.819304,1.718193,EUR 6M disc. EUR 6M curve,EUR 6M disc. EUR 6M curve,,
